In [ ]:
# Outiers, z-score future desription demo idea

In [2]:
import pandas as pd
import numpy as np

In [9]:
item = pd.read_csv('items.csv')
shops = pd.read_csv('shops.csv')
sales_train = pd.read_csv('sales_train.csv')
categories = pd.read_csv('item_categories.csv')

In [7]:
sales_train["date"] = pd.to_datetime(sales_train["date"], errors="coerce")

# Predecting future sales

In [10]:
sales_train.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day
count,1.043246e+06,1.043246e+06,1.043246e+06,1.043246e+06,1.043246e+06
mean,4.472810e+00,3.245368e+01,1.024260e+04,6.823654e+02,1.220174e+00
std,2.967065e+00,1.625856e+01,6.305360e+03,1.124829e+03,1.600203e+00
min,0.000000e+00,0.000000e+00,1.200000e+01,-1.000000e+00,-6.000000e+00
25%,2.000000e+00,2.200000e+01,4.548000e+03,1.990000e+02,1.000000e+00
50%,4.000000e+00,3.000000e+01,9.691000e+03,3.990000e+02,1.000000e+00
75%,7.000000e+00,4.700000e+01,1.561875e+04,6.993000e+02,1.000000e+00
max,1.000000e+01,5.900000e+01,2.216800e+04,5.920000e+04,2.880000e+02


In [6]:
item.describe()  # no purpose of doing this as this data doesn't explains anything

,item_id,item_category_id
count,22170.00000,22170.000000
mean,11084.50000,46.290753
std,6400.07207,15.941486
min,0.00000,0.000000
25%,5542.25000,37.000000
50%,11084.50000,40.000000
75%,16626.75000,58.000000
max,22169.00000,83.000000


In [ ]:
# Outlier: An outlier is an observation that lies an abnormal distance from other values in a random sample from
# a population

# Two graphical techniques for identifying outliers, scatter plots and box plots, along with an analytic procedure
# for detecting outliers when the distribution is normal (Grubbs' Test)
# 